"""
COMPLETELY NEW VERSION OF UNET DESIGNED FOR BRAIN TUMOUR SEGMENTATION

SPECS:

1. Input size  is 240x240x4 for each image

2. BRATS dataset 

3. DICE LOSS IS TAKEN AS A LOSS FUNCTION

4. MULTICLASS SEGMENTATION HAS BEEN IMPLEMENTED


DESC:

HERE OUR PREDICTION WILL HAVE 4 DIMENSIONS(because we have 4 classes) FOR EACH IMAGE. THESE 4 PREDICTIONS are compared with hot encoded label(Ground truth)
THIS IN A WAY TRAINS THE SYSTEM TO HOT ENCODE THE PREDICTIONS TOO.
WE ARE TRYING TO IMPLEMENT THE ABOVE STATED MODEL TO IMPLEMENT MULTICLASS SEGEMENTATION. BUT HAD TO CHANGE SOME THINGS WHICH ARE STATED BELOW

PROBLEMS AND CHANGES:

1. Faced the problem of class imbalance. So in this version we multiply dice coefficient for each class with certain weight. this weight is reciprocal of the frequency of that class

2. The problem of class imbalance still persists and dice coeff is more than 1. So i this version we have implemented a new dice coefficient function.


FUTURE:

1. IMPROVING DICE COEFFICIENT

2. 3D IMPLEMENTATION




In [1]:
'''Mounting Google Drive on the Colab notebook'''
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#file_image = '/content/gdrive/My Drive/Brain_Tumour_segmentation/Train_image.hdf5'
import h5py
#dataset has data of 484 patients. (155 images of each patient)
#data is extracted using 4 different techniques
#size of data of 1 patient is [240,240,155,4]
#for 2D segmentation we stack in 3rd dimension (axis=2)
#train_image
image_store = h5py.File("/content/gdrive/My Drive/Brain_Tumour_segmentation/Train_image.hdf5", "r")
#train_labels
label_store = h5py.File("/content/gdrive/My Drive/Brain_Tumour_segmentation/Train_label.hdf5", "r")
train_images = image_store["image"]
train_labels = label_store["label"]
#print('hi')

In [0]:
'''IMPORTING LIBRARIES'''
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
import time
import math
import os
'''Clearing tesorflow computation graph'''
tf.reset_default_graph()

In [0]:
'''DEFINING VARIABLES'''

batch_size=1              #batch size taken at a time
n_class = 4                #number of classes in the label

'''PLACEHOLDER for input and output of UNET'''
'''Here we crop the 155x240x240 image to  160x160x192 by repeating the last layer 5 times to convert 155 to 160'''
X = tf.placeholder(shape=[None,160,160,192,4], dtype=tf.float32, name='input_image')
y = tf.placeholder(shape=[None,160,160,192,1], dtype=tf.int64, name='hot_encode_label')


'''Batch variable exraction from h5py file (used by functions 'rnadom_h5py_batch' and 'test_batch')'''
out_img = np.empty((240,240,batch_size*155,4),dtype=np.float32)
out_label = np.empty((240,240,batch_size*155,1),dtype=np.int64)

DEFINING ALL THE REQUIRED FUNCTIONS

In [0]:
'''COMPUTATION GRAPH Function Definitions'''


def left_filter_def(ker_size,in_chan,out_chan,name='left_filter'):
    '''Defining a filter variable to perform convolution'''
    stddev = np.sqrt(4/(ker_size*ker_size*ker_size*ker_size*in_chan*out_chan))           #HE initialization
    if stddev < 0.0008:
        stddev = 0.0008
    return (tf.Variable(tf.truncated_normal([ker_size,ker_size,ker_size,in_chan,out_chan],stddev=stddev),name=name))


def right_filter_def(ker_size,in_chan,out_chan,name='right_filter'):
    '''Defining a filter variable to perform transpose convolution'''
    stddev = np.sqrt(4/(ker_size*ker_size*ker_size*ker_size*in_chan*out_chan))
    if stddev < 0.0008:
        stddev = 0.0008
    return (tf.Variable(tf.truncated_normal([ker_size,ker_size,ker_size,out_chan,in_chan],stddev=stddev),name=name))


def Conv_layer(input_im,filter_mask,stride,activation='None',name='conv'):
    '''Function to perform Convolution and apply activation filter'''
    '''Convolution'''
    conv = tf.nn.conv3d(input_im,filter_mask,strides = [1,stride,stride,stride,1], padding = "SAME",name=name)
    #norm_conv = tf.layers.batch_normalization(conv, training=training, momentum=0.9)
    '''Activation'''
    if activation == 'relu':
        return(tf.nn.relu(conv))
    elif activation == 'softmax':
        return(tf.nn.softmax(conv,axis=-1))
    elif activation == 'elu':
        return(tf.nn.elu(conv))
    else:
        #activation == 'None'
        return(conv)
    

def Deconv_layer(input_im,filter_mask,stride,activation='None',name='De_conv'):
    '''Function to perform Transpose Convolution and apply activation filter'''
    '''Transpose Convolution'''
    inp_shape = np.shape(input_im) #tf.shape()
    out_shape = [batch_size]+[int(inp_shape[1].value*2), int(inp_shape[2].value*2),int(inp_shape[3].value*2), int(inp_shape[4].value/2)]
    
    conv = tf.nn.conv3d_transpose(input_im, filter_mask, out_shape, strides = [1,stride,stride,stride,1], padding = "SAME",name=name)
    #norm_conv = tf.layers.batch_normalization(conv, training=training, momentum=0.9)
    '''Activation'''
    if activation == 'relu':
        return(tf.nn.relu(conv))
    elif activation == 'softmax':
        return(tf.nn.softmax(conv,axis=-1))
    elif activation == 'elu':
        return(tf.nn.elu(conv))
    else:
        #activation == 'None'
        return(conv)




In [0]:
'''Functions for batch Extraction and pre processing'''

def normalizing_input():
    '''normalization of each input channels'''
    global out_img
    '''CHANNEL INFO'''
    # maximum value found using function called "Finding_maximum_to_normalise "
    #'''max value for dimension 4 is 5337.0'''
    #'''max value for dimension 3 is 11737.0'''
    #'''max value for dimension 2 is 9751.0'''
    #'''max value for dimension 1 is 6476.0'''
    out_img[:,:,:,0] = out_img[:,:,:,0]/6476.0
    out_img[:,:,:,1] = out_img[:,:,:,1]/9751.0
    out_img[:,:,:,2] = out_img[:,:,:,2]/11737.0
    out_img[:,:,:,3] = out_img[:,:,:,3]/5337.0
    
    

def crop_image_fit_brain(in_image):
    '''cropping size was found using the code finding_brain.ipynb'''
    left = 19
    right= 210
    top  = 38
    bot  = 199
    out_image = in_image[top:(bot-1),left:(right+1),:,:]
    return (out_image)


def Pre_processing_3D(a):
    '''Function to roll axis to convert array into[depth,width,height,channels] and the divide it in batches'''
    #print(np.shape(a))
    b = np.rollaxis(a,2, 0)
    #print(np.shape(b))
    #image shape
    out_arr = np.empty(shape=[batch_size,160,np.shape(b)[1],np.shape(b)[2],np.shape(b)[3]])
    for i in range(batch_size):
        start = i*155
        end = start+155
        out_arr[i,0:155,:,:,:] = b[start:end,:,:,:]
    
    a = [out_arr[:,154:155,:,:,:]]*5
    out_arr[:,155:160,:,:,:] = a[0]
    
    '''clippig data from front of each batch'''
    '''to fit the model we remove first 3 slices of each batch'''
    '''shape of a is [batch_size,depth,width,height,channels]'''
    out_send = out_arr[:,:,:,:,:]
    #print("arr",np.shape(out_arr),"send",np.shape(out_send))
    return (out_send)

def random_rotate(in_image,in_label):
    
    check = np.random.random(1)[0]
    if check<0.25:
        out_image = in_image[:,:,::-1,:,:]
        out_lab = in_label[:,:,::-1,:,:]
        
    elif check<0.50:
        
        out_image = in_image[:,:,:,::-1,:]
        out_lab = in_label[:,:,:,::-1,:]
        
    elif check<0.75:
        
        out_image = in_image[:,::-1,:,:,:]
        out_lab = in_label[:,::-1,:,:,:]
        
    else:
        out_image =in_image
        out_lab = in_label
    
    return (out_image,out_lab)



def random_h5py_batch(current_batch_no,permute_mat):
    '''Function to take batches randomly'''
    global out_img
    global out_label
    
    '''training info'''
    train_info = 384  #100 patients with 155 images each

    if current_batch_no == 0:
        no_of_batches = train_info//batch_size  
        permute_mat = np.random.permutation(no_of_batches)
    
    start = permute_mat[current_batch_no]*batch_size*155
    end = start + (batch_size*155)
    train_images.read_direct(out_img,np.s_[:,:,start:end,:])
    train_labels.read_direct(out_label,np.s_[:,:,start:end,:])
    current_batch_no += 1
    #print(len(out_img))
    '''Input normalization'''
    normalizing_input()
    '''normalization oof labels'''
    #out_label = out_label
    #normalizing_label()
    '''converting multi class to dual class'''
    #out_label = convert_dual_class(out_label)
    '''cropping image and labels'''
    crop_out_image = crop_image_fit_brain(out_img)
    crop_out_label = crop_image_fit_brain(out_label)
    '''Rolling axes'''
    #out_img_send = np.rollaxis(crop_out_image,2, 0)
    '''hot encoding'''
    #out_label_send = crop_out_label
    '''3D conversion'''
    out_img_send = Pre_processing_3D(crop_out_image)
    out_label_send = Pre_processing_3D(crop_out_label)
    '''Data augmentation rotation'''
    #out_img_send,out_label_send = random_rotate(out_img_send,out_label_send)
     
    last=0
    if current_batch_no == len(permute_mat):
        last=1
    
    return (out_img_send,out_label_send,current_batch_no,permute_mat,last)

def test_batch():
    '''Function to take next test batch''' 
    global out_img
    global out_label
    
    '''training and testing info'''
    train_info = 380  #100 patients with 155 images each
    test_info = 484-train_info  #100 patients with 155 images each
    
    no_of_batches = test_info//batch_size  
    permute_mat = np.random.permutation(no_of_batches)
    start = (permute_mat[0]*batch_size*155) +(train_info*155)
    end = start + (155*batch_size)
    train_images.read_direct(out_img,np.s_[:,:,start:end,:])
    train_labels.read_direct(out_label,np.s_[:,:,start:end,:])
    
    '''normalization'''
    normalizing_input()
    #normalizing_label()
    '''croping images and labels'''
    crop_out_image = crop_image_fit_brain(out_img)
    crop_out_label = crop_image_fit_brain(out_label) 
    '''3D processing'''
    out_img_send = Pre_processing_3D(crop_out_image)
    out_label_send = Pre_processing_3D(crop_out_label)
    
    return (out_img_send,out_label_send)



In [0]:

  
def hot_encode(check_image,depth=n_class,name='hot_encode'):
    '''function for hot encoding images'''
    a = tf.one_hot(indices = check_image, depth=depth,name=name)
    b = tf.transpose(a,perm=[0,1,2,3,5,4])
    return b


#############################################################################################################################
#GENERALIZED DICE LOSS FUNCTION
#############################################################################################################################
def generalized_dice_coeff(y_true, y_pred):
    Ncl = y_pred.shape[-1]
    w = np.zeros(shape=(Ncl,))
    w = tf.reduce_sum(y_true, axis=[0,1,2,3]) + 1
    w = 1/((w**2))
    print(np.shape(w))
    # Compute gen dice coef:
    numerator = y_true*y_pred
    denominator = y_true+y_pred
    a=b=0
    for i in range(np.shape(w)[0]):
        a += w[i]*numerator[:,:,:,:,i]
        b += w[i]*denominator[:,:,:,:,i]
            
    num = tf.reduce_sum(a)
    den = tf.reduce_sum(b)

    gen_dice_coef = tf.subtract(1.0,tf.divide((2*num),den),name='dice_loss')

    return (gen_dice_coef)
  
#############################################################################################################################
#GENERALIZED FOCAL LOSS FUNCTION
#############################################################################################################################
def generalized_focal_loss(y_true, y_pred, gamma=2.0, alpha=0.25):
    
    Ncl = y_pred.shape[-1]
    w = np.zeros(shape=(Ncl,))
    w = tf.reduce_sum(y_true, axis=[0,1,2,3]) + 1
    w = 1/((w))
    y_pred = y_pred + 0.000000001                                               #to ensure that logarithm in next step doenst give math error
    
    ce = tf.multiply(y_true, -tf.log(y_pred))                                   #cross entropy (multiclass)
    fl_var = tf.multiply(y_true, tf.pow(tf.subtract(1., y_pred), gamma))        #focal loss variables (gamma*(1-pt)*(graund_truth))
    #fl = tf.multiply(alpha, tf.multiply(fl_var, ce))
    fl = tf.multiply(fl_var, ce)
    #fl=ce
    normalized_focal = 0
    
    for i in range(np.shape(w)[0]):
        #print(i)
        a = w[i]#/tf.reduce_sum(w)
        b = fl[:,:,:,:,i]
        #c = (b)/(tf.reduce_max(b)+1)
        normalized_focal += tf.reduce_sum(a*b)
        #b += w[i]*denominator[:,:,:,i]
    weighted_focal = tf.divide(normalized_focal,4.0,name='focal_loss')
    return (weighted_focal)

#############################################################################################################################
#MEAN SQUARE ERROR(MSE) FUNCTION
#############################################################################################################################
def diff_error(y_true, y_pred):
    error = tf.square(y_true-y_pred)
    #error = (error+0.00000000001-tf.reduce_min(error))/(tf.reduce_max(error)+0.0000001-tf.reduce_min(error))
    #a = np.shape(error)#/(1.0)
    #size = batch_size*a[1]*a[2]*a[3]*a[4]
    #size = 19660800.0
    #error = tf.divide(tf.reduce_mean(error),size, name='mse_loss')
    return tf.reduce_mean(error, name='mse_loss')

In [0]:
def find_hybrid_loss(y_true,y_pred,gamma=0.6,alpha=0.5):
    dice = generalized_dice_coeff(y_true, y_pred)
    focal = generalized_focal_loss(y_true, y_pred)
    MeanAndSquare = diff_error(y_true, y_pred)
    hybrid = tf.identity((gamma*dice)+((1-gamma)*((alpha*focal)+((1-alpha)*MeanAndSquare))),name='hybrid_loss')
    return hybrid,dice,focal,MeanAndSquare

INITIALIZING THE MODEL FILTERS

In [9]:
'''MODEL1 Filter definition'''
'''LEFT'''

filter1 = left_filter_def(3,4,8,name='filter1')
filter2 = left_filter_def(3,8,8,name='filter2')

filter3 = left_filter_def(3,8,16,name='filter3')
filter4 = left_filter_def(3,16,16,name='filter4')

filter5 = left_filter_def(3,16,32,name='filter5')
filter6 = left_filter_def(3,32,32,name='filter6')

filter7 = left_filter_def(3,32,64,name='filter7')
filter8 = left_filter_def(3,64,64,name='filter8')

filter9 = left_filter_def(3,64,128,name='filter9')
filter10= left_filter_def(3,128,64,name='filter10')

'''RIGHT'''

filter11 = right_filter_def(3,128,64,name='filter11')
filter12 = left_filter_def(3,64,64,name='filter12')
filter13 = left_filter_def(3,64,32,name='filter13')

filter14 = right_filter_def(3,64,32,name='filter14')
filter15 = left_filter_def(3,32,32,name='filter15')
filter16 = left_filter_def(3,32,16,name='filter16')

filter17 = right_filter_def(3,32,16,name='filter17')
filter18 = left_filter_def(3,16,16,name='filter18')
filter19 = left_filter_def(3,16,8,name='filter19')

filter20 = right_filter_def(3,16,8,name='filter20')
filter21 = left_filter_def(3,8,8,name='filter21')
filter22 = left_filter_def(3,8,n_class,name='filter22')

Instructions for updating:
Colocations handled automatically by placer.


DEFINING THE MODEL STRUCTURE

In [0]:
def predict_model1(X):
    '''Function to define the UNET model'''
    with tf.name_scope("BLOCK1"):
        '''BLOCK1'''
        CNN1 = Conv_layer(X,filter1,stride=1,activation='relu',name='CNN1')
        print ("CNN1",np.shape(CNN1))
        CNN2 = Conv_layer(CNN1,filter2,stride=1,activation='relu',name='CNN2')
        print ("CNN2",np.shape(CNN2))
        pool1 = tf.nn.max_pool3d(CNN2,ksize=[1,2,2,2,1],strides=[1,2,2,2,1],padding='VALID',name='POOL1')
    
    with tf.name_scope("BLOCK2"):
        '''BLOCK2'''
        CNN3 = Conv_layer(pool1,filter3,stride=1,activation='relu',name='CNN3')
        print ("CNN3",np.shape(CNN3))
        CNN4 = Conv_layer(CNN3,filter4,stride=1,activation='relu',name='CNN4')
        print ("CNN4",np.shape(CNN4))
        pool2 = tf.nn.max_pool3d(CNN4,ksize=[1,2,2,2,1],strides=[1,2,2,2,1],padding='VALID',name='POOL2')
    
    with tf.name_scope("BLOCK3"):
        '''BLOCK3'''
        CNN5 = Conv_layer(pool2,filter5,stride=1,activation='relu',name='CNN5')
        print ("CNN5",np.shape(CNN5))
        CNN6 = Conv_layer(CNN5,filter6,stride=1,activation='relu',name='CNN6')
        print ("CNN6",np.shape(CNN6))
        pool3 = tf.nn.max_pool3d(CNN6,ksize=[1,2,2,2,1],strides=[1,2,2,2,1],padding='VALID',name='POOL3')
    
    with tf.name_scope("BLOCK4"):
        '''BLOCK4'''
        CNN7 = Conv_layer(pool3,filter7,stride=1,activation='relu',name='CNN7')
        print ("CNN7",np.shape(CNN7))
        CNN8 = Conv_layer(CNN7,filter8,stride=1,activation='relu',name='CNN8')
        print ("CNN8",np.shape(CNN8))
        pool4 = tf.nn.max_pool3d(CNN8,ksize=[1,2,2,2,1],strides=[1,2,2,2,1],padding='VALID',name='POOL4')
    
    with tf.name_scope("BLOCK5"):
        '''BLOCK5'''
        CNN9 = Conv_layer(pool4,filter9,stride=1,activation='relu',name='CNN9')
        print ("CNN9",np.shape(CNN9))
        CNN10 = Conv_layer(CNN9,filter10,stride=1,activation='relu',name='CNN10')
        print ("CNN10",np.shape(CNN10))
    
    '''Moving UP'''
    
    with tf.name_scope("BLOCK6"):
        '''BLOCK6'''
        concat1 = tf.concat([CNN10,pool4],axis=4,name='CONCAT1')
        print ("concat1",np.shape(concat1))
        DCNN1= Deconv_layer(concat1,filter11,stride=2,activation='relu',name='DE_CONV1')
        print ("DCNN1",np.shape(DCNN1))
        CNN11 = Conv_layer(DCNN1,filter12,stride=1,activation='relu',name='CNN11')
        #print ("CNN11",np.shape(CNN11))
        CNN12 = Conv_layer(CNN11,filter13,stride=1,activation='relu',name='CNN12')
        #print ("CNN12",np.shape(CNN12))
    
    with tf.name_scope("BLOCK7"):
        '''BLOCK7'''
        concat2 = tf.concat([CNN12,pool3],axis=4,name='CONCAT2')
        #print ("concat2",np.shape(concat2))
        DCNN2= Deconv_layer(concat2,filter14,stride=2,activation='relu',name='DE_CONV2')
        #print ("DCNN2",np.shape(DCNN2))
        CNN13 = Conv_layer(DCNN2,filter15,stride=1,activation='relu',name='CNN13')
        #print ("CNN13",np.shape(CNN13))
        CNN14 = Conv_layer(CNN13,filter16,stride=1,activation='relu',name='CNN14')
        #print ("CNN14",np.shape(CNN14))
    
    with tf.name_scope("BLOCK8"):
        '''BLOCK8'''
        concat3 = tf.concat([CNN14,pool2],axis=4,name='CONCAT3')
        #print ("concat3",np.shape(concat3))
        DCNN3= Deconv_layer(concat3,filter17,stride=2,activation='relu',name='DE_CONV3')
        #print ("DCNN3",np.shape(DCNN3))
        CNN15 = Conv_layer(DCNN3,filter18,stride=1,activation='relu',name='CNN14')
        #print ("CNN15",np.shape(CNN15))
        CNN16 = Conv_layer(CNN15,filter19,stride=1,activation='relu',name='CNN15')
        #print ("CNN16",np.shape(CNN16))
        
    with tf.name_scope("BLOCK9"):
        '''BLOCK9'''
        concat4 = tf.concat([CNN16,pool1],axis=4,name='CONCAT4')
        #print ("concat4",np.shape(concat4))
        DCNN4= Deconv_layer(concat4,filter20,stride=2,activation='relu',name='DE_CONV4')
        #print ("DCNN4",np.shape(DCNN4))
        CNN17 = Conv_layer(DCNN4,filter21,stride=1,activation='relu',name='CNN17')
        #print ("CNN17",np.shape(CNN17))
        CNN18 = Conv_layer(CNN17,filter22,stride=1,activation='relu',name='CNN18')
        print ("CNN18",np.shape(CNN18))
    return (CNN18)

In [0]:
def chose_train_restore(learning_rate =0.0001,n_epochs = 100):
    output_dir = "/content/gdrive/My Drive/Brain_Tumour_segmentation/3D segmentation/saving_model"
    model_checkpoint_file_base = os.path.join(output_dir, "model.ckpt")

    
    if not os.path.exists(model_checkpoint_file_base + ".meta"):
        '''FIRST TIME TRAINING'''
        print("Making new")
        brand_new = True
        
        prediction = predict_model1(X)#logits
        '''----------------------------------------------------------------------------------------------------------------------------------------------------------------'''
        with tf.name_scope("LOSS_FUNCTION"):
            '''using multi dimensional dice'''
            hot_y = hot_encode(y)
            #dice = 1+ dice_coef_multilabel(hot_y,prediction)     #dice loss for verison 1
            #dice = generalized_dice_coeff(hot_y[:,:,:,:,0], prediction)
            #focal = generalized_focal_loss(hot_y[:,:,:,:,0], prediction)
            hybrid,dice,focal,sep = find_hybrid_loss(hot_y[:,:,:,:,:,0], prediction, gamma=0.9,alpha=0.8)
            #mean_error = diff_error(hot_y[:,:,:,:,0], prediction)
        '''----------------------------------------------------------------------------------------------------------------------------------------------------------------'''
        with tf.name_scope("COST_FUNCTION"):
            '''Cost function''''''Remember to change max to min min to mx depending on loss function'''
            loss = tf.reduce_mean(hybrid, name="loss")

        saver = tf.train.Saver()
        
    else:
        '''RESTORED MODEL'''
        print("Reloading existing")
        brand_new = False
        saver = tf.train.import_meta_graph(model_checkpoint_file_base + ".meta")
        g = tf.get_default_graph()
        
        sep = g.get_tensor_by_name("LOSS_FUNCTION/mse_loss:0")
        dice = g.get_tensor_by_name("LOSS_FUNCTION/dice_loss:0")
        focal = g.get_tensor_by_name("LOSS_FUNCTION/focal_loss:0")
        hybrid = g.get_tensor_by_name("LOSS_FUNCTION/hybrid_loss:0")
        prediction = g.get_tensor_by_name("BLOCK9/Softmax:0") 
        loss = g.get_tensor_by_name("COST_FUNCTION/loss:0")
        
        #X = g.get_tensor_by_name("input_image:0")
        #y = g.get_tensor_by_name("hot_encode_label:0")


    
    
    
    '''TRAINING'''
    '''starting session'''
    gpu_option = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_option)) as sess:
        '''Initializing optimizer'''
        if brand_new:
            optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
            init = tf.global_variables_initializer()
            sess.run(init)
            tf.add_to_collection("optimizer", optimizer)
        else:
            saver = tf.train.Saver()
            saver.restore(sess, model_checkpoint_file_base)
            optimizer = tf.get_collection("optimizer")[0]

        for epoch in range(9,n_epochs):
            current_batch_no = 0
            permute_mat = 0
            iteration = 0
            while(1):
                #with tf.device('/cpu:0'):
                epoch_x,epoch_y,current_batch_no,permute_mat,last = random_h5py_batch(current_batch_no,permute_mat)
                sess_results = sess.run(optimizer, feed_dict={X: epoch_x, y: epoch_y})
                #print ("epoch",epoch+1,"batch",iteration+1)#,"Cost",sess_results[0])
                
                '''DICE Coefficient for iteration'''
                #with tf.device('/cpu:0'):
                if iteration%10==0:
                    acc_train = 1-(hybrid.eval(feed_dict={X: epoch_x, y: epoch_y}))
                    hybrid_train,dice_train,focal_train,diff_train = sess.run([hybrid,dice,focal,sep], feed_dict={X: epoch_x, y: epoch_y})
                    #test_images, test_labels = test_batch()
                    #acc_test = 1-(hybrid.eval(feed_dict={X: test_images, y: test_labels}))
                    #print("Minibatch at","Epoch", epoch+1,"batch",iteration+1, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
                    print("After Epoch", epoch+1, "Hybrid Train accuracy:", 1-hybrid_train, "Dice Train accuracy:", 1-dice_train, "Focal Train accuracy:", 1-focal_train, "MSE Train accuracy:", 1-diff_train)
                if last ==1:
                    break
                iteration +=1
            test_images, test_labels = test_batch()
            hybrid_test,dice_test,focal_test,diff_test = sess.run([hybrid,dice,focal,sep], feed_dict={X: test_images, y: test_labels})
            #diff_test = sess.run(mean_error, feed_dict={X: test_images, y: test_labels})
            print("-------------------------------------------------------------------------------------------------------")
            print("After Epoch", epoch+1, "Hybrid Test accuracy:", 1-hybrid_test, "Dice Test accuracy:", 1-dice_test, "Focal Test accuracy:", 1-focal_test, "MSE Test accuracy:", 1-diff_test)
            #print("mse",1-diff_test)
            print("-------------------------------------------------------------------------------------------------------")
            '''saving model after each epoch'''
            save_path = tf.train.Saver(max_to_keep=1).save(sess, model_checkpoint_file_base)
            
            if epoch % 1 == 0:
                test_example =   test_images
                test_example_gt = test_labels#np.rollaxis(test_labels,2,0)
                sess_results = sess.run(prediction,feed_dict={X:test_example})

                sess_results = sess_results[0,100,:,:,1] + (2*sess_results[0,100,:,:,2]) + (3*sess_results[0,100,:,:,3])
                test_example = test_example[0,100,:,:,3]
                test_example_gt = test_example_gt[0,100,:,:,:]
                
                plt.figure()
                plt.imshow(np.squeeze(test_example),cmap='gray')
                plt.axis('off')
                plt.title('Original Image')
                plt.savefig('/content/gdrive/My Drive/Brain_Tumour_segmentation/3D segmentation/result/'+str(epoch)+"a_Original_Image.png")
                 
                plt.figure()
                plt.imshow(np.squeeze(test_example_gt),cmap='gray')
                plt.axis('off')
                plt.title('Ground Truth Mask')
                plt.savefig('/content/gdrive/My Drive/Brain_Tumour_segmentation/3D segmentation/result/'+str(epoch)+"b_Original_Mask.png")

                plt.figure()
                plt.imshow(np.squeeze(sess_results),cmap='gray')
                plt.axis('off')
                plt.title('Generated Mask')
                plt.savefig('/content/gdrive/My Drive/Brain_Tumour_segmentation/3D segmentation/result/'+str(epoch)+"c_Generated_Mask.png")

                plt.close('all')

    

COMMENTS ON DICE LOSS AND ACCURACY

1. Under the name_scope "LOSS FUNCTION", the variable named dice corresponds to dice loss and since the function dice_multipleclass() returns a value between -1 and 0(both included) , we add 1. Also another reason for this is there is no maximize function in adam optimizer(or any other optimizing function).

2. While printing  the accuracy (everywhere)  we have to print dice coefficient and not dice loss therefore we add 1 to the dice_loss calculation

In [17]:
chose_train_restore()

Making new
CNN1 (?, 160, 160, 192, 8)
CNN2 (?, 160, 160, 192, 8)
CNN3 (?, 80, 80, 96, 16)
CNN4 (?, 80, 80, 96, 16)
CNN5 (?, 40, 40, 48, 32)
CNN6 (?, 40, 40, 48, 32)
CNN7 (?, 20, 20, 24, 64)
CNN8 (?, 20, 20, 24, 64)
CNN9 (?, 10, 10, 12, 128)
CNN10 (?, 10, 10, 12, 64)
concat1 (?, 10, 10, 12, 128)
DCNN1 (1, 20, 20, 24, 64)
CNN18 (1, 160, 160, 192, 4)
(4,)
After Epoch 10 Hybrid Train accuracy: -1.3168888092041016 Dice Train accuracy: 6.258487701416016e-06 Focal Train accuracy: -16.648681640625 MSE Train accuracy: 0.7500009089708328
After Epoch 10 Hybrid Train accuracy: -1.0222878456115723 Dice Train accuracy: 6.794929504394531e-05 Focal Train accuracy: -12.966863632202148 MSE Train accuracy: 0.7500047832727432


KeyboardInterrupt: ignored